In [1]:
import torch

def load_data():
    import PIL.Image as Image
    import numpy as np
    import os
    
    xs = []
    ys = []
    
    for filename in os.listdir('data/cifar10'):
        if filename.endswith('.jpg'):
            img = Image.open(f'data/cifar10/%s' % filename)
            img = torch.FloatTensor(np.array(img))/255
            # 把通道放到前面 3, 32, 32
            img= img.permute(2, 0, 1)
            y = filename.split('_')[0]
            
            xs.append(img)
            ys.append(int(y))

    return xs, ys

xs, ys = load_data()
xs[0].shape, ys[0],len(xs), len(ys)

(torch.Size([3, 32, 32]), 0, 60000, 60000)

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, xs, ys):
        self.xs = xs
        self.ys = ys
    
    def __getitem__(self, i):
        return self.xs[i], self.ys[i]
    
    def __len__(self):
        return len(self.xs)
    
dataset = Dataset(xs, ys)
len(dataset), dataset[0][0].shape, dataset[0][1]

(60000, torch.Size([3, 32, 32]), 0)

In [3]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True,drop_last=True)

data = next(iter(dataloader))
data[0].shape, data[1].shape, data[1]

(torch.Size([64, 3, 32, 32]),
 torch.Size([64]),
 tensor([6, 6, 7, 6, 8, 2, 0, 6, 9, 9, 2, 2, 8, 3, 3, 5, 1, 4, 5, 4, 7, 5, 1, 0,
         9, 7, 5, 1, 4, 9, 5, 2, 3, 1, 8, 1, 2, 9, 2, 7, 3, 9, 1, 4, 3, 9, 4, 2,
         4, 0, 0, 6, 9, 4, 8, 3, 3, 3, 9, 4, 4, 2, 4, 5]))

In [4]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 3, 32, 32 => 16, 32, 32
        # Output Size = (32 + 2 * padding - kernel_size)/stride + 1 = (32- 3)/1 + 1 = 32
        self.conv1 = torch.nn.Conv2d(3, 16, 3, padding=1)
        # 16, 32, 32 => 32, 14, 14
        # (32 + 2 * 0 - 5) / 2 + 1 = 14
        self.conv2 = torch.nn.Conv2d(16, 32, 5, 2, padding=0)
        # 32, 7, 7 => 128, 1, 1
        # (7 + 2 * 0 - 7) / 1 + 1 = 1
        self.conv3 = torch.nn.Conv2d(32, 128, 7, padding=0)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.relu = torch.nn.ReLU()
        self.rc = torch.nn.Linear(128, 10)
        
    def forward(self, x):
        # 3, 32, 32 => 16, 32, 32
        x = self.conv1(x)
        x = self.relu(x)
        # 16, 32, 32 => 32, 14, 14
        x = self.conv2(x)
        x = self.relu(x)
        # 32, 14, 14 => 32, 7, 7
        x = self.pool(x)
        # 32, 7, 7 => 128, 1, 1
        x = self.conv3(x)
        x = self.relu(x)
        # 128, 1, 1 => 128
        x = x.view(-1, 128)
        # 128 => 10
        x = self.rc(x)
        return x
    
model = Model()
model(torch.randn(64,3, 32, 32)).shape

torch.Size([64, 10])

In [6]:
def train():
    # cpu 3m 3s 245ms
    # gpu 21s 452ms
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = Model().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = torch.nn.CrossEntropyLoss()
    for epoch in range(10):
        for i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                acc = (y_pred.argmax(1) == y).float().mean()
                print(epoch, i, loss.item(), acc.item())
                
    torch.save(model, 'model/3.model')
    
train()

cuda
0 0 2.2947757244110107 0.234375
0 100 1.8015269041061401 0.34375
0 200 1.6162450313568115 0.375
0 300 1.7082611322402954 0.359375
0 400 1.4501310586929321 0.46875
0 500 1.3312181234359741 0.46875
0 600 1.5734888315200806 0.359375
0 700 1.4916167259216309 0.40625
0 800 1.1122463941574097 0.546875
0 900 1.2071938514709473 0.578125
1 0 1.2324661016464233 0.625
1 100 1.322042465209961 0.53125
1 200 1.0986336469650269 0.625
1 300 1.1536554098129272 0.625
1 400 1.3185017108917236 0.53125
1 500 1.141427755355835 0.65625
1 600 1.1750624179840088 0.609375
1 700 1.0907058715820312 0.59375
1 800 1.2704440355300903 0.5625
1 900 1.2426044940948486 0.578125
2 0 1.111155390739441 0.59375
2 100 0.9162545204162598 0.640625
2 200 1.1196144819259644 0.671875
2 300 0.9551293253898621 0.640625
2 400 1.0592290163040161 0.625
2 500 1.041473388671875 0.625
2 600 1.103433609008789 0.5625
2 700 1.0977317094802856 0.671875
2 800 1.1878944635391235 0.578125
2 900 0.9284736514091492 0.671875
3 0 1.00658988952

In [10]:
@torch.no_grad()
def predict():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load('model/3.model', map_location=device)
    model.eval()
    
    total = 0
    correct = 0
    
    for i in range(1000):
        x,y = next(iter(dataloader))
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        # print(y_pred.shape) torch.Size([64, 10])
        correct += (y_pred.argmax(1) == y).float().sum().item()
        total += len(y)
    print(correct, total)
    print(correct/total)
    
predict()

48158.0 64000
0.75246875


In [12]:
@torch.no_grad()
def predict():
    device = torch.device("cpu")
    model = torch.load('model/3.model', map_location=device)
    model = model.to(device)
    model.eval()

    total = 0
    correct = 0

    for i in range(1000):
        x, y = next(iter(dataloader))
        # x, y = x.to(device), y.to(device)  # 将数据移动到 CPU
        y_pred = model(x)
        correct += (y_pred.argmax(1) == y).float().sum().item()
        total += len(y)

    accuracy = correct / total
    print(f"Correct predictions: {correct}, Total samples: {total}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

predict()

Correct predictions: 48115.0, Total samples: 64000
Accuracy: 75.18%
